# Imports

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from torch import optim
import torch.nn as nn


# ---
import sys; sys.path.append('../')

from commons.imgs_mean_std import *

from commons.dataset import *
from commons.imageutils import *
from commons.HistCollection import *

from modules.img_transforms import *
from modules.train_functions import *
from modules.Comparator import *

from modules.Dataset import *
from modules.EarlyStopper import *

from modules.Model import *

# Constants & hyperparams

In [2]:
RANDOM_STATE=None

IMAGE_SIZE=220

BATCH_SIZE=32
NUM_EPOCHS=100

LEARNING_RATE=.001
MOMENTUM=.95

EARLY_PATIENCE=20
SCHEDULER_PATIENCE=10

# CUDA

In [3]:
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

if device == 'cuda':
    print(torch.cuda.is_available())
    print(torch.cuda.get_device_name(0))
    print('Devices:', torch.cuda.device_count())

2.5.1+cu124
Device: cuda


# Loading datasets

In [4]:
train_dataset = pd.read_csv(os.path.join(DATASETS_PATHS.norm_faces.info, 'train_dataset.csv'))
val_dataset = pd.read_csv(os.path.join(DATASETS_PATHS.norm_faces.info, 'val_dataset.csv'))
test_dataset = pd.read_csv(os.path.join(DATASETS_PATHS.norm_faces.info, 'test_dataset.csv'))

# Creating custom datasets

In [5]:
normalization= normalize(IMAGE_SIZE, MEAN, STD)

train_dataset = CD_TrippletsCreator(train_dataset, transform=normalization, data_augmentation_tranforms=[data_augmentation()])
val_dataset = CD_TrippletsCreator(val_dataset, transform=normalization)
test_dataset = CD_TrippletsCreator(test_dataset, transform=normalization)

# Dataloaders

In [6]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, drop_last=True)

# Early stopping

In [7]:
def early_callback(**kwarks):
    print(" > Early Stop <")

early_stopper = EarlyStopper(EARLY_PATIENCE, .001, callback=early_callback, verbose=True)

# Loading model

In [8]:
model = Model()
if device == 'cuda' and torch.cuda.device_count() > 1: model = nn.DataParallel(model)

model.to(device)

Model(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 256, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 512, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runn

# Defining tools

In [9]:
criterion = nn.TripletMarginWithDistanceLoss(distance_function=dst).to(device)

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=.00001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=.1, patience=SCHEDULER_PATIENCE, verbose=True)

c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


# Training

In [ ]:
def callback(**kwargs):
	print(f"Epoch: {kwargs['epoch']} =================================")
	print(f"\ttrain_loss: {kwargs['train_loss']:2f}\tval_loss: {kwargs['val_loss']:2f}")
	print(f"\tdst_mean_train (pos, neg): ({np.mean(kwargs['train_h'].posit_dst[-1]):2f}, {np.mean(kwargs['train_h'].negat_dst[-1]):2f}) dst_mean_val (pos, neg): ({np.mean(kwargs['val_h'].posit_dst[-1]):2f}, {np.mean(kwargs['val_h'].negat_dst[-1]):2f})")


train_h, val_h = train_loop(model, train_loader, val_loader=val_loader,
                            optimizer=optimizer, criterion=criterion, dst=dst, num_epoch=100, device=device,
                            early_stopper=early_stopper, scheduler=scheduler, callback=callback)

model.load_state_dict(early_stopper.best_model)

Epoch: 1 =================================
	train_loss: 1.013602	val_loss: 0.752264
	dst_mean_train (pos, neg): (3.364281, 3.363387) dst_mean_val (pos, neg): (0.789065, 1.060162)
Best model currently!!


 25%|██▌       | 4/16 [01:48<05:15, 26.30s/it]

In [11]:
save_model(model, 'model.pth', complete=False)

# Model evaluation

In [12]:
def train_val_analisys(**kwargs):

	train_posit_mean_h = [np.mean(elm) for elm in  kwargs['train_h'].posit_dst]
	train_negat_mean_h = [np.mean(elm) for elm in  kwargs['train_h'].negat_dst]

	train_posit_median_h = [np.median(elm) for elm in kwargs['train_h'].posit_dst]
	train_negat_median_h = [np.median(elm) for elm in kwargs['train_h'].negat_dst]

	val_posit_mean_h = [np.mean(elm) for elm in  kwargs['val_h'].posit_dst]
	val_negat_mean_h = [np.mean(elm) for elm in  kwargs['val_h'].negat_dst]

	val_posit_median_h = [np.median(elm) for elm in kwargs['val_h'].posit_dst]
	val_negat_median_h = [np.median(elm) for elm in kwargs['val_h'].negat_dst]


	plt.figure(figsize=(24, 6))

	plt.subplot(1, 3, 1); plt.title('Train / Val loss')
	plt.xlabel('Epochs')
	plt.ylabel('Loss')
	plt.xticks([1, len(kwargs['val_h'].loss)])
	plt.plot(kwargs['train_h'].loss, label='Train')
	plt.plot(kwargs['val_h'].loss, label='Val')
	plt.plot(kwargs['best_loss'][0]-1, kwargs['best_loss'][1], 'o', )
	plt.annotate('best', (kwargs['best_loss'][0]-1, kwargs['best_loss'][1]))
	plt.axvline(kwargs['best_loss'][0]-1, linestyle='dashed', color='red', linewidth=1)
	plt.legend()

	plt.subplot(1, 3, 2); plt.title('Distances means')
	plt.xlabel('Epochs')
	plt.ylabel('Mean')
	plt.xticks([1, len(kwargs['val_h'].loss)])
	plt.plot(train_posit_mean_h, label='Train positive')
	plt.plot(train_negat_mean_h, label='Train negative')
	plt.plot(val_posit_mean_h, label='Val positive')
	plt.plot(val_negat_mean_h, label='Val negative')
	plt.axvline(kwargs['best_loss'][0]-1, linestyle='dashed', color='red', linewidth=1)
	plt.legend()

	plt.subplot(1,3,3); plt.title('Distances medians')
	plt.xlabel('Epoch')
	plt.ylabel('Median')
	plt.xticks([1, len(kwargs['val_h'].loss)])
	plt.plot(train_posit_median_h, label='Train positive')
	plt.plot(train_negat_median_h, label='Train negative')
	plt.plot(val_posit_median_h, label='Val positive')
	plt.plot(val_negat_median_h, label='Val negative')
	plt.axvline(kwargs['best_loss'][0]-1, linestyle='dashed', color='red', linewidth=1)
	plt.legend()


In [ ]:
train_val_analisys(train_h=train_h, val_h=val_h, best_loss=(early_stopper.best_epoch, early_stopper.best_loss))

# Test

In [ ]:
test_h = eval(model, test_loader, device=device, criterion=criterion, dst=dst)
print('Loss', test_h.loss)
print(f'Posit dst:\t| mean {np.mean(test_h.posit_dst)}\t| median {np.median(test_h.posit_dst)}\t| std {np.std(test_h.posit_dst)}\t| min, max {np.min(test_h.posit_dst)}, {np.max(test_h.posit_dst)}\t| diff(min,max) {np.max(test_h.posit_dst)-np.min(test_h.posit_dst)}')
print(f'Negat dst:\t| mean {np.mean(test_h.negat_dst)}\t| median {np.median(test_h.negat_dst)}\t| std {np.std(test_h.negat_dst)}\t| min, max {np.min(test_h.negat_dst)}, {np.max(test_h.negat_dst)}\t| diff(min,max) {np.max(test_h.negat_dst)-np.min(test_h.negat_dst)}')
print(f'Diff(max(posit_dst), min(negat_dst)) {min(test_h.negat_dst) - max(test_h.posit_dst)}')

# Segundo entrenamiento con el mejor modelo actual

In [15]:
train_h_, val_h_ =  train_h.copy(), val_h.copy()
train_h_.loss, val_h_.loss = train_h_.loss[:early_stopper.best_epoch], val_h_.loss[:early_stopper.best_epoch]
train_h_.posit_dst, val_h_.posit_dst  = train_h_.posit_dst[:early_stopper.best_epoch], val_h_.posit_dst[:early_stopper.best_epoch]
train_h_.negat_dst, val_h_.negat_dst  = train_h_.negat_dst[:early_stopper.best_epoch], val_h_.negat_dst[:early_stopper.best_epoch]
train_h_.asetid, val_h_.asetid  = train_h_.asetid[:early_stopper.best_epoch], val_h_.asetid[:early_stopper.best_epoch]
train_h_.nsetid, val_h_.nsetid = train_h_.nsetid[:early_stopper.best_epoch], val_h_.nsetid[:early_stopper.best_epoch]

In [ ]:
early_stopper.reset()

train_h_, val_h_ = train_loop(model, train_loader, val_loader=val_loader, initial_epoch=early_stopper.best_epoch,
                            optimizer=optimizer, criterion=criterion, dst=dst, num_epoch=100, device=device,
                            early_stopper=early_stopper, scheduler=scheduler, callback=callback, train_h=train_h_, val_h=val_h_)

model.load_state_dict(early_stopper.best_model)
save_model(model, 'model.pth', complete=False)

In [22]:
model.load_state_dict(early_stopper.best_model)
save_model(model, 'model.pth', complete=False)

# Analisys again

In [ ]:
train_val_analisys(train_h=train_h_, val_h=val_h_, best_loss=(early_stopper.best_epoch, early_stopper.best_loss))

In [ ]:
test_h = eval(model, test_loader, device=device, criterion=criterion, dst=dst)
print('Loss', test_h.loss)
print(f'Posit dst:\t| mean {np.mean(test_h.posit_dst)}\t| median {np.median(test_h.posit_dst)}\t| std {np.std(test_h.posit_dst)}\t| min, max {np.min(test_h.posit_dst)}, {np.max(test_h.posit_dst)}\t| diff(min,max) {np.max(test_h.posit_dst)-np.min(test_h.posit_dst)}')
print(f'Negat dst:\t| mean {np.mean(test_h.negat_dst)}\t| median {np.median(test_h.negat_dst)}\t| std {np.std(test_h.negat_dst)}\t| min, max {np.min(test_h.negat_dst)}, {np.max(test_h.negat_dst)}\t| diff(min,max) {np.max(test_h.negat_dst)-np.min(test_h.negat_dst)}')
print(f'Diff(max(posit_dst), min(negat_dst)) {min(test_h.negat_dst) - max(test_h.posit_dst)}')